# 02_DiD_estimates.ipynb

In [1]:
from differences import ATTgt
import figure_utilities
from matplotlib import pyplot as plt
plt.rcParams["figure.dpi"] = 300
plt.rcParams['savefig.dpi'] = 300
import numpy as np
import os
import pandas as pd


# Store paths.
INPUT_DATA = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/restricted.csv"
OUTPUT_FIGURES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/DiD/figures"
OUTPUT_TABLES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/DiD/tables"

# Read restricted sample panel dataset into memory.
df = pd.read_csv(INPUT_DATA)

# Drop rows where voluntary dismissal occurred. 
df = df.loc[df['voluntary_dismissal'] == 0, :]

In [2]:
# Store a list of column names containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]

# Create dictionaries to map between string represetation and integer representation of each month.
month_to_int_dictionary = {key: value for value, key in enumerate(value_vars)}
int_to_month_dictionary = {key: value for key, value in enumerate(value_vars)}

In [3]:
# Drop rows for which we are missing any Zestimates.
has_all_zestimates_mask = df[value_vars].notna().all(axis=1) 
print(f"Limiting sample to {has_all_zestimates_mask.sum()} evictions for which we observe Zestimates at every month "
      "from 2012-12 to 2022-12.")
df = df.loc[has_all_zestimates_mask, :]

Limiting sample to 2657 evictions for which we observe Zestimates at every month from 2012-12 to 2022-12.


In [4]:
# Reshape from wide to long.
df = pd.melt(df,
             id_vars=['case_number', 'latest_docket_date', 'judgment_for_plaintiff'],
             value_vars=value_vars, var_name='month', value_name='zestimate')
df = df.sort_values(by=['case_number', 'month'])

# Get 'first treated' date for each observation.
df.loc[:, 'latest_docket_date'] = pd.to_datetime(df['latest_docket_date']).dt.strftime('%Y-%m')

# Convert months from string format to integer format.
df.loc[:, 'month'] = df['month'].replace(month_to_int_dictionary)
df.loc[:, 'latest_docket_date'] = df['latest_docket_date'].replace(month_to_int_dictionary)
never_treated_mask = (df['judgment_for_plaintiff'] == 0)
df.loc[never_treated_mask, 'latest_docket_date'] = np.NaN

# Set index to entity and time.
df = df.set_index(['case_number', 'month'])

1    235466
0     86031
Name: judgment_for_plaintiff, dtype: int64


In [5]:
# Run DiD.
att_gt = ATTgt(data=df, cohort_name='latest_docket_date', freq='M')
result = att_gt.fit(formula='zestimate', control_group='never_treated', n_jobs=-1)

Computing ATTgt [workers=10]  100%|████████████████████| 5040/5040 [04:16<00:00,


In [6]:
# Get cohort-aggregated ATT(t)s.
results_df = att_gt.aggregate('event')
results_df = results_df.loc[-10:35]
results_df.columns = results_df.columns.droplevel().droplevel()
    
# Plot event study-style plot of ATTs.
fig, ax = plt.subplots()
x = results_df.index
y = results_df['ATT']
y_upper = results_df['upper']
y_lower = results_df['lower']
ax.grid(True)
ax.set_xlabel("Month Relative to Treatment")
ax.set_ylabel("ATT")
ax.set_title("ATT(g, t) Estimates, Aggregated by Month Relative to Treatment")
figure_utilities.plot_labeled_vline(ax, x=0, text="Last Docket Date")
figure_utilities.plot_labeled_hline(ax, y=0, text="")
figure_utilities.plot_scatter_with_error_bars(ax,
                                              x,
                                              y,
                                              y_upper,
                                              y_lower)
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "att_gt_estimates_event_study.png"))


att_gt.plot('event')